# Домашнее задание №2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
item_weights = data.groupby('item_id')['sales_value'].sum().apply(np.log).reset_index()
item_weights['weight'] = item_weights['sales_value'] / item_weights['sales_value'].sum()
item_weights.describe()

,item_id,sales_value,weight
count,8.905100e+04,8.905100e+04,88640.0
mean,5.115772e+06,-inf,0.0
std,5.178973e+06,NaN,0.0
min,2.567100e+04,-inf,-0.0
25%,9.665830e+05,1.252763e+00,-0.0
50%,1.448516e+06,2.377693e+00,0.0
75%,9.553042e+06,3.830921e+00,0.0
max,1.802456e+07,1.305621e+01,-0.0


In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    #item_weights = item_weights.fillna(0)
    items_weights = items_weights.dropna()
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values
    weights = weights / np.sum(weights)
    recs = np.random.choice(items, size=n, replace=False)

    
    return recs.tolist()

In [7]:
%%time

# your_code
items = data_train.item_id.unique()
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))
result.head(2)

CPU times: user 5.77 s, sys: 62.1 ms, total: 5.83 s
Wall time: 5.83 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[990055, 1041984, 6552930, 1669244, 12604112]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1122142, 5569042, 1247280, 1022531, 9265791]"


In [8]:
def precision_at_k(recommended_list, bought_list, k=5):
    #recommended_list = recommended_list.strip('[]').split(', ')
    #bought_list = bought_list.strip('[]').split()
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [9]:
precision_weighted_random1 = result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()
print("Precision@5 weighted random recommendation:", precision_weighted_random1)

Precision@5 weighted random recommendation: 0.0007835455435847208


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [10]:
result = pd.read_csv('predictions_basic.csv')
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,[ 920308 926804 946489 1006718 1017061 ...,"[13416054, 936084, 7410040, 9527114, 377218]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,[ 840386 889774 898068 909714 929067 ...,"[5574336, 990072, 868548, 995880, 842226]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,[ 835098 872137 910439 924610 992977 ...,"[1277401, 94446, 3133282, 1925252, 855699]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list.strip('[]').split(', ')
    bought_list = bought_list.strip('[]').split()
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [12]:
precision_random = result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()
print(f"Precision@5 random recommendation: {precision_random}")

precision_popular = result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()
print(f"Precision@5 popular recommendation: {precision_popular}")

precision_itemitem = result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()
print(f"Precision@5 item-item recommendation: {precision_itemitem}")

precision_cosine = result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()
print(f"Precision@5 cosine similarity recommendation: {precision_cosine}")

precision_tfidf = result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()
print(f"Precision@5 TFIDF recommendation: {precision_tfidf}")

precision_own_purchases = result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()
print(f"Precision@5 own purchases recommendation: {precision_own_purchases}")


Precision@5 random recommendation: 0.0005876591576885406
Precision@5 popular recommendation: 0.15523996082272082
Precision@5 item-item recommendation: 0.033594515181195064
Precision@5 cosine similarity recommendation: 0.03525954946131262
Precision@5 TFIDF recommendation: 0.03614103819784544
Precision@5 own purchases recommendation: 0.1799869409076044


In [ ]:
precision_random = result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()
print(f"Precision@5 random recommendation: {precision_random}")

precision_popular = result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()
print(f"Precision@5 popular recommendation: {precision_popular}")

precision_itemitem = result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()
print(f"Precision@5 item-item recommendation: {precision_itemitem}")

precision_cosine = result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()
print(f"Precision@5 cosine similarity recommendation: {precision_cosine}")

precision_tfidf = result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()
print(f"Precision@5 TFIDF recommendation: {precision_tfidf}")

precision_own_purchases = result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()
print(f"Precision@5 own purchases recommendation: {precision_own_purchases}")


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [13]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [16]:
def precision_at_k2(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision


### 1.1 Random recommendation

In [17]:
def random_recommendation(items, n=5):
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    return recs.tolist()

In [19]:
items = data_train.item_id.value_counts().index[:5000]
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
precision_random = result.apply(lambda row: precision_at_k2(row['random_recommendation'], row['actual']), axis=1).mean()
print("Precision@5 random recommendation:", precision_random)

Precision@5 random recommendation: 0.005974534769833491


### 1.2 Popularity-based recommendation

In [20]:
def popularity_recommendation(data, items, n=5):
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular = popular[popular['item_id'].isin(items)]
    popular.sort_values('sales_value', ascending=False, inplace=True)
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [21]:
items = data_train.item_id.value_counts().index[:5000]
popular_recs = popularity_recommendation(data_train, items, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result['actual'] = result['actual'].apply(lambda x: list(map(int, x)))

In [22]:
precision_popular2 = result.apply(lambda row: precision_at_k2(row['popular_recommendation'], row['actual']), axis=1).mean()
print("Precision@5 popular recommendation:", precision_popular2)

Precision@5 popular recommendation: 0.15523996082272082


### 1.3 Weighted random recommender

In [23]:
def weighted_random_recommendation(items_weights, n=5):
    items_weights = items_weights.dropna()
    items = items_weights['item_id'].values
    weights = items_weights['weight'].values
    weights = weights / np.sum(weights)
    recs = np.random.choice(items, size=n, replace=False)
    return recs.tolist()

In [24]:
items = data_train.item_id.value_counts().index[:5000]
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(item_weights, n=5))

In [25]:
precision_weighted_random2 = result.apply(lambda row: precision_at_k2(row['weighted_random_recommendation'], row['actual']), axis=1).mean()
print("Precision@5 weighted random recommendation:", precision_weighted_random2)

Precision@5 weighted random recommendation: 0.0004897159647404506


### Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾

In [5]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [6]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
user_item_matrix.shape

(2499, 5001)

In [9]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/2499 [00:00<?, ?it/s]

In [ ]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во ближайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/2499 [00:00<?, ?it/s]

In [39]:
recs

(array([1766,  717, 1652, 2457, 1900], dtype=int32),
 array([ 8309., 11300., 10128., 13899., 13037.]))

In [45]:
[id_to_itemid[rec[0]] for rec in recs]


KeyError: 8309.0

In [41]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

KeyError: 8309.0

In [1]:
result.head(2)

NameError: name 'result' is not defined